### Bonus 2: TD 2 Report - Nolan Cacheux

To properly conclude this 2nd assignment and to help you understand why such a data structure is impossible to modify without causing complete corruption of the blockchain, I invite you to "mistreat" your implementation. Create situations of corruption that you find interesting and test how your blockchain reacts in these situations.

**Imagine having to sell such technology to a major client. You must convince them, through a written report in the form of a Notebook, and with concrete examples, that your technology is unbreakable.**

<font color="8888CC">Create a new Notebook for this report. To do this, in the main tab of Jupyter Notebook, in the upper right corner, there is a ```New``` button. Click on it and select the ```Python 3``` kernel.</font>

> I recommend generating 2 wallets to get interesting situations. Also, draw inspiration from what we have seen in class.

> If you want to format the text you include in this Notebook, I suggest you take inspiration from the texts in this Notebook by double-clicking on one of the texts to see the source code. At the top of the Notebook, you can change the type of a cell by replacing ```Code``` with ```Markdown```.

In [7]:
from sys import path
path.append('../scripts')
from certificate import Certificate
from wallet import Wallet
from block import Block
from blockchain import Blockchain

walletAlice = Wallet()
walletBob = Wallet()
publicKeyAlice = walletAlice.publicKey
publicKeyBob = walletBob.publicKey

certAlice1 = Certificate(publicKeyAlice)
certAlice2 = Certificate(publicKeyAlice)
certBob1 = Certificate(publicKeyBob)
certBob2 = Certificate(publicKeyBob)

walletAlice.sign(certAlice1)
walletAlice.sign(certAlice2)
walletBob.sign(certBob1)
walletBob.sign(certBob2)

blockchain = Blockchain()

latestBlock = blockchain.get_latest_block()
blockAlice = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certAlice1, certAlice2])
walletAlice.sign(blockAlice)
blockchain.blockList.append(blockAlice)

latestBlock = blockchain.get_latest_block()
blockBob = Block(publicKeyBob, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certBob1, certBob2])
walletBob.sign(blockBob)
blockchain.blockList.append(blockBob)

print("\nBlockchain initiale:")
blockchain.display()

print("\nScénario 1 : Modification de la clé publique d'un certificat")
certAlice1.issuerPublicKey = "fake_public_key"
print("Blockchain valide ?", blockchain.is_legit())

print("\nScénario 2 : Ajout d'un bloc non signé")
blockTampered = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certAlice1])
blockchain.blockList.append(blockTampered)
print("Blockchain valide ?", blockchain.is_legit())

print("\nScénario 3 : Inversion de l'ordre des blocs")
blockchain.blockList[0], blockchain.blockList[1] = blockchain.blockList[1], blockchain.blockList[0]
print("Blockchain valide ?", blockchain.is_legit())

print("\nScénario 4 : Changement du hash parent d'un bloc")
blockchain.blockList[1].parentBlockHash = "manipulated_hash"
print("Blockchain valide ?", blockchain.is_legit())



Blockchain initiale:
Block 0:
{'type': 'Block', 'issuerPublicKey': '0000000000', 'timestamp': 0, 'hash': 'a70c7e36dfd03e9493708664c2effdd5cc5721263477809856cac03793779834', 'signature': '', 'index': 0, 'parent': '0000000000000000000000000000000000000000000000000000000000000000', 'certificates': []}
Block 1:
{'type': 'Block', 'issuerPublicKey': '3082012230', 'timestamp': 1737732652931, 'hash': '01ef4db355969f7399084dbed7b9a70ea73c4e246f2230cc24d50359ab78d699', 'signature': '75cb7e39f6950284d076d80e29eeb76b17c2c48337cd65e263c0f03ace43a0822f2b3947fe8d35d588710b3dceb502b3c55c383070b52c85e474c83134e3b143cd00c4316cd84eb5f711e37825536402c5cb2a31adf5e263357a21593a4250681d1558d85d2586652a4c158b4bc3cb64352628f66054b4f24b0c2c83887d040558dde0faba257f469e00f6dba5414bae398579d11ac19f0d3f37462b799a6d2b3b0c3a1d4c649cb4bb416559abf517ef5acf54a20429de6010de73e2f7ddfaf02cd41965a98b49670d8ef09645bfffc3b1cbe1333e145d4964812c79e34bacdf82f6b68e05182ac2fbd8e83cd7ad37a446658c797bcc9ad4afc8bc37e7a75936', 'index

In [8]:
print("\nScénario 5 : Suppression d'un bloc")
blockchain.blockList.pop(1)
print("Blockchain valide ?", blockchain.is_legit())

blockchain = Blockchain()
blockchain.blockList.append(blockAlice)
blockchain.blockList.append(blockBob)

print("\nScénario 6 : Ajout d'un bloc avec un index invalide")
blockInvalidIndex = Block(publicKeyBob, 42, latestBlock.hash(), [certBob1])
walletBob.sign(blockInvalidIndex)
blockchain.blockList.append(blockInvalidIndex)
print("Blockchain valide ?", blockchain.is_legit())

print("\nScénario 7 : Ajout d'un bloc avec un certificat mal signé")
certInvalid = Certificate(publicKeyAlice)
certInvalid.issuerPublicKey = "tampered_public_key"
blockInvalidCert = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certInvalid])
walletAlice.sign(blockInvalidCert)
blockchain.blockList.append(blockInvalidCert)
print("Blockchain valide ?", blockchain.is_legit())

print("\nScénario 8 : Ajout de plusieurs blocs simultanément")
blockBatch1 = Block(publicKeyAlice, latestBlock.indexInBlockchain + 1, latestBlock.hash(), [certAlice1])
blockBatch2 = Block(publicKeyBob, latestBlock.indexInBlockchain + 2, latestBlock.hash(), [certBob2])
walletAlice.sign(blockBatch1)
walletBob.sign(blockBatch2)
blockchain.blockList.append(blockBatch1)
blockchain.blockList.append(blockBatch2)
print("Blockchain valide ?", blockchain.is_legit())

print("\nÉtat final de la blockchain:")
blockchain.display()



Scénario 5 : Suppression d'un bloc
Blockchain valide ? False

Scénario 6 : Ajout d'un bloc avec un index invalide
Blockchain valide ? False

Scénario 7 : Ajout d'un bloc avec un certificat mal signé
Blockchain valide ? False

Scénario 8 : Ajout de plusieurs blocs simultanément
Blockchain valide ? False

État final de la blockchain:
Block 0:
{'type': 'Block', 'issuerPublicKey': '0000000000', 'timestamp': 0, 'hash': 'a70c7e36dfd03e9493708664c2effdd5cc5721263477809856cac03793779834', 'signature': '', 'index': 0, 'parent': '0000000000000000000000000000000000000000000000000000000000000000', 'certificates': []}
Block 1:
{'type': 'Block', 'issuerPublicKey': '3082012230', 'timestamp': 1737732652931, 'hash': 'ff1c9cc1522b8b3226c78617864a862b9b27737968fb21c778153c773ea07d1c', 'signature': '75cb7e39f6950284d076d80e29eeb76b17c2c48337cd65e263c0f03ace43a0822f2b3947fe8d35d588710b3dceb502b3c55c383070b52c85e474c83134e3b143cd00c4316cd84eb5f711e37825536402c5cb2a31adf5e263357a21593a4250681d1558d85d258665